In [4]:
%%html
<div style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h3>Tracking AR (Esfera Guía)</h3>
    <p style="font-size: 12px; color: #aaa;">La esfera te esta siguiendo</p>
    
    <div id="threejs_container_sphere" style="position: relative; display: inline-block; width: 640px; height: 480px; border: 2px solid #555; border-radius: 10px; overflow: hidden; background: #000;">
        </div>
    
    <div style="margin-top: 15px;">
        <button id="btn_on_sphere" onclick="iniciarSphereAR()" style="padding: 10px 20px; background: #f1c40f; color: #000; border: none; border-radius: 5px; cursor: pointer; font-weight: bold;">ACTIVAR ESFERA</button>
        <button id="btn_off_sphere" onclick="detenerSphereAR()" style="padding: 10px 20px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px; display: none;">APAGAR</button>
    </div>
    
    <div style="margin-top: 10px;">
        <button id="toggle_bg_sphere" onclick="toggleBackgroundSphere()" style="padding: 8px 15px; background: #34495e; color: white; border: none; border-radius: 5px; cursor: pointer; font-size: 12px;">Mostrar Mi Cara</button>
    </div>
    
    <p id="log_sphere" style="color: #f1c40f; font-size: 13px; margin-top: 10px;">Estado: Esperando cámara...</p>
</div>

<script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/face_mesh.js" crossorigin="anonymous"></script>

<script>
let tScene, tCamera, tRenderer, tObject;
let tCameraInstance = null;
let tVideoElement = null;
let isVideoVisible = false; // Estado inicial: Video oculto

// --- FUNCIÓN CORREGIDA PARA MOSTRAR/OCULTAR VIDEO ---
function toggleBackgroundSphere() {
    // 1. Buscamos el elemento de video directamente por su ID para asegurar que existe
    const vid = document.getElementById('v_threejs_sphere');
    const btn = document.getElementById('toggle_bg_sphere');
    
    if (vid) {
        isVideoVisible = !isVideoVisible;
        // Cambiamos el estilo display entre 'block' (visible) y 'none' (oculto)
        vid.style.display = isVideoVisible ? 'block' : 'none';
        // Actualizamos el texto del botón
        if (btn) btn.innerText = isVideoVisible ? 'Ocultar Mi Cara' : 'Mostrar Mi Cara';
    } else {
        console.log("El video aún no está listo. Inicia la cámara primero.");
    }
}

// Función para crear la esfera amarilla de alambre (igual que tu imagen)
function crearEsferaGuia() {
    const geometry = new THREE.IcosahedronGeometry(30, 2); // Esfera geodésica
    const material = new THREE.MeshBasicMaterial({ 
        color: 0xffff00,       // Amarillo
        wireframe: true        // Modo alambre
    });
    return new THREE.Mesh(geometry, material);
}

// Limpieza de memoria
function detenerSphereAR() {
    if (tCameraInstance) {
        try { tCameraInstance.stop(); } catch(e) { console.log(e); }
        tCameraInstance = null;
    }
    
    if (tRenderer) { tRenderer.dispose(); tRenderer = null; }
    if (tScene) {
        while(tScene.children.length > 0) { tScene.remove(tScene.children[0]); }
        tScene = null;
    }
    
    if (window.cameraManager && window.cameraManager.currentStream) {
        window.cameraManager.currentStream.getTracks().forEach(t => t.stop());
        window.cameraManager.currentStream = null;
    }
    
    if (tVideoElement) {
        if (tVideoElement.srcObject) {
            tVideoElement.srcObject.getTracks().forEach(t => t.stop());
            tVideoElement.srcObject = null;
        }
        tVideoElement = null;
    }
    
    // Reseteamos estado del botón
    isVideoVisible = false;
    const btn = document.getElementById('toggle_bg_sphere');
    if(btn) btn.innerText = 'Mostrar Mi Cara';
    
    const container = document.getElementById('threejs_container_sphere');
    if (container) container.innerHTML = '';
    
    document.getElementById('btn_on_sphere').style.display = 'inline-block';
    document.getElementById('btn_off_sphere').style.display = 'none';
    document.getElementById('log_sphere').innerText = "Sistema apagado.";
}

async function iniciarSphereAR() {
    const log = document.getElementById('log_sphere');
    
    try {
        log.innerText = "Iniciando sistema...";
        detenerSphereAR(); // Limpieza previa
        await new Promise(resolve => setTimeout(resolve, 300));
        
        const container = document.getElementById('threejs_container_sphere');
        
        // 1. CREAMOS EL VIDEO (Oculto por defecto con display: none)
        tVideoElement = document.createElement('video');
        tVideoElement.id = 'v_threejs_sphere'; // ID importante para el botón
        tVideoElement.width = 640;
        tVideoElement.height = 480;
        tVideoElement.autoplay = true;
        tVideoElement.playsInline = true;
        tVideoElement.muted = true;
        // style.display = none inicial
        tVideoElement.style.cssText = 'position: absolute; top: 0; left: 0; transform: scaleX(-1); width: 100%; height: 100%; display: none;';
        container.appendChild(tVideoElement);
        
        // 2. CREAMOS EL CANVAS (Donde se dibuja la esfera)
        const tCanvas = document.createElement('canvas');
        tCanvas.id = 'c_threejs_sphere';
        tCanvas.width = 640;
        tCanvas.height = 480;
        // z-index alto para estar encima
        tCanvas.style.cssText = 'position: absolute; top: 0; left: 0; transform: scaleX(-1); width: 100%; height: 100%; z-index: 10;';
        container.appendChild(tCanvas);
        
        log.innerText = "Accediendo a cámara...";
        
        const stream = await navigator.mediaDevices.getUserMedia({ 
            video: { width: 640, height: 480, facingMode: 'user' } 
        });
        
        if (!window.cameraManager) window.cameraManager = {};
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'sphere_ar';
        
        tVideoElement.srcObject = stream;
        await tVideoElement.play();
        
        await new Promise(resolve => {
            if (tVideoElement.videoWidth > 0) resolve();
            else tVideoElement.onloadedmetadata = () => resolve();
        });
        
        document.getElementById('btn_on_sphere').style.display = 'none';
        document.getElementById('btn_off_sphere').style.display = 'inline-block';
        
        log.innerText = "Configurando 3D...";
        
        // ========== CONFIGURAR THREE.JS ==========
        tScene = new THREE.Scene();
        tCamera = new THREE.OrthographicCamera(-320, 320, 240, -240, 0.1, 1000);
        tCamera.position.z = 10;
        
        tRenderer = new THREE.WebGLRenderer({ 
            canvas: tCanvas, 
            alpha: true, // Fondo transparente
            antialias: true 
        });
        tRenderer.setSize(640, 480);
        
        // Agregar la Esfera Amarilla
        tObject = crearEsferaGuia();
        tObject.visible = false;
        tScene.add(tObject);
        
        log.innerText = "Activando seguimiento...";
        
        // ========== CONFIGURAR MEDIAPIPE ==========
        const faceMesh = new FaceMesh({
            locateFile: (file) => `https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/${file}`
        });
        
        faceMesh.setOptions({
            maxNumFaces: 1,
            refineLandmarks: true,
            minDetectionConfidence: 0.5,
            minTrackingConfidence: 0.5
        });
        
        faceMesh.onResults((results) => {
            if (window.cameraManager.currentFilter !== 'sphere_ar' || !tRenderer) return;
            
            tRenderer.clear();
            
            if (results.multiFaceLandmarks && results.multiFaceLandmarks.length > 0) {
                const landmarks = results.multiFaceLandmarks[0];
                const nose = landmarks[4]; // Punta de la nariz
                
                const x = (nose.x - 0.5) * 640;
                const y = -(nose.y - 0.5) * 480;
                const z = -nose.z * 200;
                
                tObject.position.set(x, y, z);
                tObject.rotation.x += 0.01;
                tObject.rotation.y += 0.02;
                
                tObject.visible = true;
                log.innerText = "Esfera activa siguiendo nariz";
            } else {
                tObject.visible = false;
                log.innerText = "Rostro no detectado";
            }
            
            tRenderer.render(tScene, tCamera);
        });
        
        // ========== INICIAR CÁMARA ==========
        tCameraInstance = new Camera(tVideoElement, {
            onFrame: async () => {
                if (window.cameraManager.currentFilter === 'sphere_ar') {
                    await faceMesh.send({image: tVideoElement});
                }
            },
            width: 640,
            height: 480
        });
        
        await tCameraInstance.start();
        
    } catch (e) {
        log.innerText = "Error: " + e.message;
        console.error(e);
        detenerSphereAR();
    }
}
</script>